# EX2: Lartillot's Model


Equation for Lartillot model

$p(\theta|d)= \frac{p(d|\theta)p(\theta)}{p(d)}$

In [4]:
install.packages("matrixStats")
install.packages("mvtnorm")

Installing package into ‘/home/avaj040/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

Installing package into ‘/home/avaj040/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)



In [8]:
set.seed(42)
library(matrixStats)
library(mvtnorm)

inflation <- 1
n = 1e3
v = 0.01
p = 1
R = 40
Rt = 2000
Re = 360
tau = exp(-7.25)
eta = exp(-7.75)

target.mean = rep(0, p)
target.var = diag(x = v/(v + 1), ncol = p, nrow = p)

log.true.c <- function(v, p) {
  (p / 2) * (log(v) - log(1 + v)) # this formula is given right after (58) in Lartillot.
}
ltrue.c = log.true.c(v = v, p = p)

###### delete later
likelihood = function(xi, v)


epanechnikov.results = triangle.results = doubleexp.results = norm.results = simulation.results = rep(NA, 300)
ptm <- proc.time()


In [12]:
# joint density of data and parameter
g = function(theta, v){
  like = -sum(theta^2) / (2 * v)
  prior = sum(dnorm(theta, 0, 1, log = T))
  return(like + prior)
}

In [15]:
# we choose to evaluate the posterior density at 0
for (i in 1:300){
  if (p == 1){
    # FI
    target.sample = rnorm(n = n, sd = sqrt(target.var))

    a = sin(R * target.sample) / target.sample
    post.dens = sum(a) / (n * pi)
    lpriorlike = g(0, v)
    simulation.results[i] = lpriorlike - log(post.dens)

    # normal-FI
    post.dens = mean(dnorm(target.sample, mean = 0, sd = 4 * tau))
    norm.results[i] = lpriorlike - log(post.dens)
    # double-exponential-FI
    post.dens = mean(dcauchy(target.sample, location = 0, scale = eta))
    doubleexp.results[i] = lpriorlike - log(post.dens)
    # triangle-kernel-FI
    a = (1 / (Rt * target.sample^2)) * (1 - cos(Rt * (target.sample)))
    post.dens = sum(a) / (n * pi^p)
    triangle.results[i] = lpriorlike - log(post.dens)
    # Epanechnikov-kernel-FI
    a = (-2 / Re) * (1 / target.sample^2) * cos(Re * (-target.sample)) + (2 / Re^2) * (1 / -target.sample^3) * sin(Re * (-target.sample))
    post.dens = sum(a) / (n * pi^p)
    epanechnikov.results[i] = lpriorlike - log(post.dens)
  }
  if (p >= 2){
    target.sample = rnorm(p * n, mean = 0, sd = sqrt(v / (v + 1))) * inflation
    target.sample = matrix(c(target.sample), nrow = n, ncol = p, byrow = T)
    a = abs(rowProds(sin(R * target.sample) / target.sample)) # multiplying together different dimensions
    post.dens = sum(a) / (n * pi^p)
    lpriorlike = g(rep(0, p), v = v)
    simulation.results[i] = lpriorlike - log(post.dens)

    # normal-FI
    post.dens = mean(dmvnorm(target.sample, mean = rep(0, p), sigma = diag(2 * tau, nrow = p)))
    lpriorlike = g(0, v = v)
    norm.results[i] = lpriorlike - log(post.dens)
  }
  print(paste0("iteration", i))
}
proc.time() - ptm
mean(is.na(simulation.results))
mean(simulation.results)
sd(simulation.results) / sqrt(300)
square.diff = (simulation.results - rep(ltrue.c, 300))^2
(1 / 300) * sum(square.diff)

png("GFIcase4_R.png")
plot(density(simulation.results), xlab = "estimates of marginal likelihood",
     main = "The Fourier Integral Estimates")
abline(v = ltrue.c, col = "red")
mtext(substitute(paste("R = ", v),
                 list(v = R)),
      side = 1, line = 4, col = "blue")
#legend("topright", legend = c("density of estimates", "true value"), col = c("black", "red"), lwd = 2, lty = 1)

plot(density(norm.results), xlab = "estimates of marginal likelihood",
     main = "The normal-kernel Fourier Integral Estimates")
abline(v = ltrue.c, col = "red")
mtext(substitute(paste("tau = ", v),
                 list(v = tau)),
      side = 1, line = 4, col = "blue")

plot(density(doubleexp.results), xlab = "estimates of marginal likelihood",
     main = "The Cauchy-kernel Fourier Integral Estimates")
abline(v = ltrue.c, col = "red")
mtext(substitute(paste("eta = ", v),
                 list(v = eta)),
      side = 1, line = 4, col = "blue")

plot(density(triangle.results), xlab = "estimates of marginal likelihood",
     main = "The triangular-kernel Fourier Integral Estimates")
abline(v = ltrue.c, col = "red")
mtext(substitute(paste("R = ", v),
                 list(v = Rt)),
      side = 1, line = 4, col = "blue")

plot(density(epanechnikov.results), xlab = "estimates of marginal likelihood",
     main = "The Epanechnikov-kernel Fourier Integral Estimates")
abline(v = ltrue.c, col = "red")
mtext(substitute(paste("R = ", v),
                 list(v = Re)),
      side = 1, line = 4, col = "blue")
dev.off()

mean(norm.results)
sd(norm.results) / sqrt(300)
mean(doubleexp.results)
sd(doubleexp.results) / sqrt(300)
mean(triangle.results)
sd(triangle.results) / sqrt(300)
mean(epanechnikov.results)
sd(epanechnikov.results) / sqrt(300)



[1] "iteration1"
[1] "iteration2"
[1] "iteration3"
[1] "iteration4"
[1] "iteration5"
[1] "iteration6"
[1] "iteration7"
[1] "iteration8"
[1] "iteration9"
[1] "iteration10"
[1] "iteration11"
[1] "iteration12"
[1] "iteration13"
[1] "iteration14"
[1] "iteration15"
[1] "iteration16"
[1] "iteration17"
[1] "iteration18"
[1] "iteration19"
[1] "iteration20"
[1] "iteration21"
[1] "iteration22"
[1] "iteration23"
[1] "iteration24"
[1] "iteration25"
[1] "iteration26"
[1] "iteration27"
[1] "iteration28"
[1] "iteration29"
[1] "iteration30"
[1] "iteration31"
[1] "iteration32"
[1] "iteration33"
[1] "iteration34"
[1] "iteration35"
[1] "iteration36"
[1] "iteration37"
[1] "iteration38"
[1] "iteration39"
[1] "iteration40"
[1] "iteration41"
[1] "iteration42"
[1] "iteration43"
[1] "iteration44"
[1] "iteration45"
[1] "iteration46"
[1] "iteration47"
[1] "iteration48"
[1] "iteration49"
[1] "iteration50"
[1] "iteration51"
[1] "iteration52"
[1] "iteration53"
[1] "iteration54"
[1] "iteration55"
[1] "iteration56"
[

   user  system elapsed 
  0.604   0.020 120.926 

[1] 0

[1] -2.303341

[1] 0.00260495

...

[1] 0.01776151

[1] -2.293077

[1] 0.01300003

[1] -2.304288

[1] 0.007021475

![GFIcase4_R.png](GFIcase4_R.png)